# BWT for Linear String

There are many tutorials in the internet.
I'll not repeat it here.

## ToC
0. Setup reference
1. Sort suffix array
2. Get BWT string
3. Calculate BWT occurances
4. Calculate start position of each character in suffix array 
5. Search string
6. BWT with error quota
7. Prefix doubling for sorting suffix arrays (Not required)

(The last two parts may be new for you)

## 0. Setup reference

In [1]:
ref = "abaabaabbaa"  # change this

# print
print("Reference sequences:", ref)

# initialize
ref += "$"               # add end of line character
leng = len(ref)          # length of reference
chrs = sorted(set(ref))  # all possible characters (sorted)

# print
print("Index Suffix")
for i in range(len(ref)):
    print(f"{i:5d} {ref[i:]}")

Reference sequences: abaabaabbaa
Index Suffix
    0 abaabaabbaa$
    1 baabaabbaa$
    2 aabaabbaa$
    3 abaabbaa$
    4 baabbaa$
    5 aabbaa$
    6 abbaa$
    7 bbaa$
    8 baa$
    9 aa$
   10 a$
   11 $


## 1. Sort Suffix Array

We sort the array in less efficient way (compared to prefix doubling), but clearer.

In [2]:
sa = sorted(range(leng), key=lambda i: ref[i:])

# print
print("Rank Index suffix")
for i, s in enumerate(sa):
    print(f"{i:4d} {s:5d} {ref[s:]}")

Rank Index suffix
   0    11 $
   1    10 a$
   2     9 aa$
   3     2 aabaabbaa$
   4     5 aabbaa$
   5     0 abaabaabbaa$
   6     3 abaabbaa$
   7     6 abbaa$
   8     8 baa$
   9     1 baabaabbaa$
  10     4 baabbaa$
  11     7 bbaa$


## 2. Get BWT string

BWT string = the previous character of the start position in suffix array

Or you can rotate the string and retrieve BWT from the last character

In [3]:
bwt = [ref[i- 1] for i in sa]

# print
print(f"Rank SA_index {'suffix':{leng}s} BWT")
for i, s in enumerate(sa):
    print(f"{i:4d} {s:8d} {ref[s:]}{ref[:s]} {bwt[i]}")

Rank SA_index suffix       BWT
   0       11 $abaabaabbaa a
   1       10 a$abaabaabba a
   2        9 aa$abaabaabb b
   3        2 aabaabbaa$ab b
   4        5 aabbaa$abaab b
   5        0 abaabaabbaa$ $
   6        3 abaabbaa$aba a
   7        6 abbaa$abaaba a
   8        8 baa$abaabaab b
   9        1 baabaabbaa$a a
  10        4 baabbaa$abaa a
  11        7 bbaa$abaabaa a


## 3. Calculate BWT occurances

The saving of occurances of each character can
speed up the string matching when using the LF mapping property

Note: I have additional first row that all the character are 0.

In [4]:
occurs = [{i: 0 for i in chrs}]  # list of occurances
for i in bwt:
    occurs.append(dict(occurs[-1]))  # copy previous occurances
    occurs[-1][i] += 1

# print
def formatIndex():
    strs = [f"Rank SA_index SA BWT   {'   '.join(chrs)}"]
    for i, s in enumerate(sa):
        strs.append(f"{i:4d} {s:8d} {ref[s]:>2s} {bwt[i]:>3s} ")
        strs[-1] += " ".join(f"{occurs[i][ch]:3d}" for ch in chrs)
    strs.append(" " * 21 + " ".join(f"{occurs[-1][ch]:3d}" for ch in chrs))
    return strs

for s in formatIndex():
    print(s)

Rank SA_index SA BWT   $   a   b
   0       11  $   a   0   0   0
   1       10  a   a   0   1   0
   2        9  a   b   0   2   0
   3        2  a   b   0   2   1
   4        5  a   b   0   2   2
   5        0  a   $   0   2   3
   6        3  a   a   1   2   3
   7        6  a   a   1   3   3
   8        8  b   b   1   4   3
   9        1  b   a   1   4   4
  10        4  b   a   1   5   4
  11        7  b   a   1   6   4
                       1   7   4


## 4. Calculate Suffix Array start position of each character

In [5]:
pos = 0
count = {}
for ch in chrs:  # all characters are sorted
    count[ch] = pos
    pos += occurs[-1][ch]  # The last row of occurances = character's count

# print
print(f"{count=}")
strs = formatIndex()
print("Start " + strs[0])
for i, s in enumerate(strs[1:]):
    if i < len(sa) and count[ref[sa[i]]] == i:
        print(ref[sa[i]] + " --> " + s)
    else:
        print("      " + s)

count={'$': 0, 'a': 1, 'b': 8}
Start Rank SA_index SA BWT   $   a   b
$ -->    0       11  $   a   0   0   0
a -->    1       10  a   a   0   1   0
         2        9  a   b   0   2   0
         3        2  a   b   0   2   1
         4        5  a   b   0   2   2
         5        0  a   $   0   2   3
         6        3  a   a   1   2   3
         7        6  a   a   1   3   3
b -->    8        8  b   b   1   4   3
         9        1  b   a   1   4   4
        10        4  b   a   1   5   4
        11        7  b   a   1   6   4
                             1   7   4


## 5. Search String

Using LF mapping property

In [6]:
target = "aab"  # change this
print("Search:", target)


ps, pe = 0, leng - 1  # The answer is between ps and pe(included)

for pos_rev, ch in enumerate(reversed(target)):
    ps_rank = occurs[ps][ch]          # LF mapping index
    pe_rank = occurs[pe + 1][ch] - 1
    ch_pos = count[ch]
    new_ps = ch_pos + ps_rank        # Shift to character's start position
    new_pe = ch_pos + pe_rank

    # print
    print(f"Find {target}")
    print(f"    " + " " * (len(target) - pos_rev) + "^")
    print(f"Before: {ps=} {pe=}")
    strs = formatIndex()
    print(f"Pos   " + strs[0])
    for i, s in enumerate(strs[1:]):
        if i == ps or i == pe:
            print(f"{i:2d} -> ", end="")
        else:
            print("      ", end="")
        print(s, end="")
        if i == ps:
            print(f" -> {ch}: {ps_rank:2d}(occurance) + {ch_pos:2d}(start) = {new_ps}")
        elif i == pe:
            print(f" -> {ch}: {pe_rank:2d}(occurance) + {ch_pos:2d}(start) = {new_pe}")
        else:
            print()

    # replace ps, pe
    ps = new_ps
    pe = new_pe
    print(f"After: {ps=} {pe=}\n")
    
    # no suffix included in [ps, pe] = not found
    if ps > pe:
        raise ValueError(f"Cannot found {target}")


print()
print(f"Possible start position(SA_index) that match {target} in {ref}")
print(f"Rank SA_index {'Suffix':{leng}s}")
for i in range(ps, pe + 1):
    print(f"{i:4d} {sa[i]:8d} {ref[sa[i]:]:{leng}s}")

Search: aab
Find aab
       ^
Before: ps=0 pe=11
Pos   Rank SA_index SA BWT   $   a   b
 0 ->    0       11  $   a   0   0   0 -> b:  0(occurance) +  8(start) = 8
         1       10  a   a   0   1   0
         2        9  a   b   0   2   0
         3        2  a   b   0   2   1
         4        5  a   b   0   2   2
         5        0  a   $   0   2   3
         6        3  a   a   1   2   3
         7        6  a   a   1   3   3
         8        8  b   b   1   4   3
         9        1  b   a   1   4   4
        10        4  b   a   1   5   4
11 ->   11        7  b   a   1   6   4 -> b:  3(occurance) +  8(start) = 11
                             1   7   4
After: ps=8 pe=11

Find aab
      ^
Before: ps=8 pe=11
Pos   Rank SA_index SA BWT   $   a   b
         0       11  $   a   0   0   0
         1       10  a   a   0   1   0
         2        9  a   b   0   2   0
         3        2  a   b   0   2   1
         4        5  a   b   0   2   2
         5        0  a   $   0   2   3
    

## 6. Search String with error quota

Use recursion to try all possible characters in each position

In [7]:
def searchString(targ, pos, ps, pe, quota=1):
    # no char to find -> success
    if pos == -1:
        return targ

    # BWT core
    # first item = original character
    # second and more items = use quota to change the character
    for ti, ch in enumerate([targ[pos], *sorted(set(chrs) - set(targ[pos] + "$"))]):
        # current target
        targ = targ[:pos] + ch + targ[pos + 1:]
        
        # same as previous
        ps_occur = occurs[ps][ch]
        pe_occur = occurs[pe + 1][ch] - 1
        ch_pos = count[ch]
        new_ps = ch_pos + ps_occur
        new_pe = ch_pos + pe_occur

        # print quota and current target
        print()
        print(f"Find {target}  Quota remain: {quota - (ti != 0)}")
        if target != targ:  
            print(f"Try  {targ}")
        print(f"     " + " " * pos + "^")
        
        # same
        print(f"Before: {ps=} {pe=}")
        strs = formatIndex()
        print(f"Pos   " + strs[0])
        for i, s in enumerate(strs[1:]):
            if i == ps or i == pe:
                print(f"{i:2d} -> ", end="")
            else:
                print("      ", end="")
            print(s, end="")
            if i == ps:
                print(f" -> {ch}: {ps_occur:2d}(occurance) + {ch_pos:2d}(start) = {new_ps}")
            elif i == pe:
                print(f" -> {ch}: {pe_occur:2d}(occurance) + {ch_pos:2d}(start) = {new_pe}")
            else:
                print()
        print(f"After: ps={new_ps} pe={new_pe}", end=" ")

        # fail
        # 1 -> no quota for changing character -> exit this recursion
        # 2 -> continue the loop
        if new_ps > new_pe:
            print("-> not found", end=' ')
            if quota == 0:
                print("-> no quota -> go to previous step")
                raise ValueError(f"Cannot found {target}, error quota excceed")
            print("-> use quota")
            continue

        # success -> next one
        #   if ti != 0 -> this character is changed by using one quota
        #   if no result found in pos -1 -> try another character
        try:
            print("-> next")
            return searchString(targ, pos - 1, new_ps, new_pe, quota if ti == 0 else quota - 1)
        except ValueError:
            pass

    # All possible characters are tried
    raise ValueError(f"Cannot found {target}. All possible characters are tried")


target = "babb"  # change this
err_quota = 1    # change this
print("Search string:", target)
print("With error quota:", err_quota)
targ = searchString(target, len(target) - 1, 0, leng - 1, err_quota)
print("\nPossible string:", targ)

Search string: babb
With error quota: 1

Find babb  Quota remain: 1
        ^
Before: ps=0 pe=11
Pos   Rank SA_index SA BWT   $   a   b
 0 ->    0       11  $   a   0   0   0 -> b:  0(occurance) +  8(start) = 8
         1       10  a   a   0   1   0
         2        9  a   b   0   2   0
         3        2  a   b   0   2   1
         4        5  a   b   0   2   2
         5        0  a   $   0   2   3
         6        3  a   a   1   2   3
         7        6  a   a   1   3   3
         8        8  b   b   1   4   3
         9        1  b   a   1   4   4
        10        4  b   a   1   5   4
11 ->   11        7  b   a   1   6   4 -> b:  3(occurance) +  8(start) = 11
                             1   7   4
After: ps=8 pe=11 -> next

Find babb  Quota remain: 1
       ^
Before: ps=8 pe=11
Pos   Rank SA_index SA BWT   $   a   b
         0       11  $   a   0   0   0
         1       10  a   a   0   1   0
         2        9  a   b   0   2   0
         3        2  a   b   0   2   1
       

## 7. Sort Suffix Array by prefix doubling (step 1)

Prefix doubling will be used in graph BWT.
But we can also learn this in linear one.

Prefix doubling contains two steps
1. Sort by character (length = 1)
2. Sort by two ranks from prvious steps (length = 2^n, n>=1)

In [8]:
# step1
chr_rank = {r: i for i, r in enumerate(chrs)}  # chr -> rank
rank = [chr_rank[r] for r in ref]
prefix_leng = 1

# step1 print
print(f"Index {'Prefix':{leng}s} {'Suffix':{leng}s} Rank")
for i in sorted(range(leng), key=rank.__getitem__):
    print(f"{i:5d} {ref[i:i+prefix_leng]:{leng}s} {ref[i:]:{leng}s} {rank[i]} = {rank[i]}")

# step2
while prefix_leng <= leng:
    # the rank of 0-2^n
    # the rank of 2^n-2^n+1
    new_rank = [(rank[i], rank[i + prefix_leng] if i + prefix_leng < leng else -1) for i in range(leng)]
    # tuple[rank, rank] -> rank
    rank_rank = {r: i for i, r in enumerate(sorted(set(new_rank)))}
    rank = [rank_rank[r] for r in new_rank]
    prefix_leng *= 2
    
    # step2 print
    print(f"Prefix Length = {prefix_leng}")
    print(f"Index {'Prefix':{leng}s} {'Suffix':{leng}s} Rank")
    for i in sorted(range(leng), key=rank.__getitem__):
        print(f"{i:5d} {ref[i:i+prefix_leng]:{leng}s} {ref[i:]:{leng}s} ({new_rank[i][0]:2d}, {new_rank[i][1]:2d}) = {rank[i]}")

# rank (index: position, value=rank) -> 
# suffix array (index=rank, value=position)
sa = [0] * leng
for i, r in enumerate(rank):
    sa[r] = i
    
# print (same as step2 format)
print("Rank Index suffix")
for i, s in enumerate(sa):
    print(f"{i:4d} {s:5d} {ref[s:]}")

Index Prefix       Suffix       Rank
   11 $            $            0 = 0
    0 a            abaabaabbaa$ 1 = 1
    2 a            aabaabbaa$   1 = 1
    3 a            abaabbaa$    1 = 1
    5 a            aabbaa$      1 = 1
    6 a            abbaa$       1 = 1
    9 a            aa$          1 = 1
   10 a            a$           1 = 1
    1 b            baabaabbaa$  2 = 2
    4 b            baabbaa$     2 = 2
    7 b            bbaa$        2 = 2
    8 b            baa$         2 = 2
Prefix Length = 2
Index Prefix       Suffix       Rank
   11 $            $            ( 0, -1) = 0
   10 a$           a$           ( 1,  0) = 1
    2 aa           aabaabbaa$   ( 1,  1) = 2
    5 aa           aabbaa$      ( 1,  1) = 2
    9 aa           aa$          ( 1,  1) = 2
    0 ab           abaabaabbaa$ ( 1,  2) = 3
    3 ab           abaabbaa$    ( 1,  2) = 3
    6 ab           abbaa$       ( 1,  2) = 3
    1 ba           baabaabbaa$  ( 2,  1) = 4
    4 ba           baabbaa$     ( 2,  1) = 4
  